In [21]:
from elexon import ElexonRawClient
from calendar import monthrange
import datetime
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm
from sys import exit
from tqdm import tqdm
import os 

In [22]:
filepath = "0.DataSource/NordPool/DA_Hourly"
DA_TurnOver = pd.read_excel(f"{filepath}/AuctionVolumeAndPrices.xlsx",sheet_name='TurnoverVolume')
DA_Prices = pd.read_excel(f"{filepath}/AuctionVolumeAndPrices.xlsx",sheet_name='Prices')
DA_TurnOver['Date'] = pd.to_datetime(DA_TurnOver['Date'])
DA_Prices['Date'] = pd.to_datetime(DA_Prices['Date'])

In [23]:
DA_TurnOver

,Date,1,2,3A,3B,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,2015-01-01,14917.0,14582.1,14127.2,NaN,13317.8,13916.3,14539.6,13954.7,14300.6,...,16296.2,16334.1,17309.2,17425.0,17578.1,17237.2,16337.4,15586.4,15012.9,15787.4
1,2015-01-02,14680.6,14305.5,14480.8,NaN,14574.2,14450.7,14674.9,14796.7,14927.6,...,16786.5,17207.0,18600.9,19364.7,19749.0,19525.5,19136.0,17434.3,16553.3,15689.4
2,2015-01-03,13707.7,13178.8,12550.1,NaN,12136.0,11722.8,11641.7,11517.8,12671.6,...,16657.9,16580.3,17030.7,17978.8,18665.0,18399.6,18928.7,17202.6,15922.9,15206.5
3,2015-01-04,13560.5,13342.5,12955.5,NaN,12727.1,12210.9,11509.3,11465.7,12062.2,...,16554.6,17169.6,17835.3,19365.7,19551.8,19291.7,18984.9,17887.0,16354.4,15373.1
4,2015-01-05,15693.7,15244.8,14667.0,NaN,14248.9,13823.2,13587.9,13804.6,16903.4,...,18891.9,18939.6,19353.1,20575.1,20490.6,20449.5,19887.5,18160.9,16924.9,15822.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403,2024-04-26,9690.6,9226.9,9085.7,NaN,9126.0,9166.8,9739.7,10697.3,12798.2,...,11886.1,11768.3,12100.4,12821.6,14188.1,14747.0,15597.3,14708.0,13937.3,12084.2
3404,2024-04-27,10968.9,9736.6,9545.7,NaN,9101.5,9128.9,9054.6,9277.5,9681.1,...,10335.9,10583.0,10665.4,11107.8,11722.4,11889.7,12390.4,12246.0,11253.5,11229.6
3405,2024-04-28,10104.8,9449.2,9355.9,NaN,9033.0,8940.3,8938.0,8840.8,8996.8,...,10190.0,10167.9,10172.5,10729.9,11346.5,12048.1,11692.1,11384.2,11106.3,10381.6
3406,2024-04-29,9450.7,9460.1,9461.9,NaN,9174.8,9184.3,9268.6,9762.1,10831.9,...,12172.0,12207.4,11630.8,12198.0,12787.4,13749.7,13176.8,12034.0,11648.3,11051.2


In [24]:
DA_Prices

,Date,1,2,3A,3B,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,2015-01-01,40.79,39.61,37.18,NaN,32.40,29.29,27.24,24.91,25.04,...,43.94,41.00,36.81,49.44,64.91,49.44,45.60,46.02,40.32,36.30
1,2015-01-02,29.51,27.94,25.50,NaN,20.92,17.38,19.85,20.67,25.36,...,39.82,36.54,36.52,47.97,80.00,56.72,37.13,44.52,37.09,36.86
2,2015-01-03,35.77,35.94,35.65,NaN,33.02,32.00,31.72,35.93,35.95,...,41.59,36.57,40.65,57.92,74.34,57.13,42.03,44.02,44.04,36.58
3,2015-01-04,40.01,40.01,51.46,NaN,36.07,35.38,32.29,32.10,34.00,...,49.90,38.51,37.02,56.69,79.98,54.92,45.96,41.00,40.98,35.91
4,2015-01-05,36.58,36.55,36.07,NaN,35.50,34.29,30.43,35.18,38.51,...,37.48,37.06,37.06,54.97,90.73,58.51,42.43,36.58,38.98,36.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403,2024-04-26,70.52,63.92,63.21,NaN,63.21,62.98,64.63,72.44,89.41,...,65.39,64.09,65.38,75.70,83.96,97.78,99.52,88.86,78.05,75.84
3404,2024-04-27,76.00,75.39,75.04,NaN,74.00,68.68,66.97,66.01,65.90,...,68.19,62.04,64.44,67.34,78.91,86.00,83.92,79.95,78.86,70.04
3405,2024-04-28,78.71,78.75,82.38,NaN,72.68,61.05,55.51,53.98,61.86,...,65.21,49.98,48.04,62.52,70.50,78.93,85.76,84.01,79.50,65.25
3406,2024-04-29,62.62,56.07,56.22,NaN,55.00,56.71,59.00,69.54,89.00,...,42.62,42.59,48.98,59.94,73.09,88.71,93.43,90.72,77.31,62.68


In [26]:
def SearchColumns(Year,Month,Day):
    Typical = [1,2,'3A',4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    MarchCO = [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    OctCO = [1,2,'3A','3B',4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    #
    if (Year==2024) and (Month==3) and (Day==31): SearchCol = MarchCO
    elif (Year==2024) and (Month==10) and (Day==27): SearchCol = OctCO
    elif (Year==2023) and (Month==3) and (Day==26): SearchCol = MarchCO
    elif (Year==2023) and (Month==10) and (Day==29): SearchCol = OctCO
    elif (Year==2022) and (Month==3) and (Day==27): SearchCol = MarchCO
    elif (Year==2022) and (Month==10) and (Day==30): SearchCol = OctCO
    elif (Year==2021) and (Month==3) and (Day==28): SearchCol = MarchCO
    elif (Year==2021) and (Month==10) and (Day==31): SearchCol = OctCO
    elif (Year==2020) and (Month==3) and (Day==29): SearchCol = MarchCO
    elif (Year==2020) and (Month==10) and (Day==25): SearchCol = OctCO
    elif (Year==2019) and (Month==3) and (Day==31): SearchCol = MarchCO
    elif (Year==2019) and (Month==10) and (Day==27): SearchCol = OctCO
    elif (Year==2018) and (Month==3) and (Day==25): SearchCol = MarchCO
    elif (Year==2018) and (Month==10) and (Day==28): SearchCol = OctCO
    elif (Year==2017) and (Month==3) and (Day==26): SearchCol = MarchCO
    elif (Year==2017) and (Month==10) and (Day==29): SearchCol = OctCO
    elif (Year==2016) and (Month==3) and (Day==27): SearchCol = MarchCO
    elif (Year==2016) and (Month==10) and (Day==30): SearchCol = OctCO
    elif (Year==2015) and (Month==3) and (Day==29): SearchCol = MarchCO
    elif (Year==2015) and (Month==10) and (Day==25): SearchCol = OctCO    
    else:
        SearchCol = Typical
    return SearchCol

In [32]:
'''
Years = tqdm(range(2015,2024+1))
DataFrame = pd.DataFrame(columns=["Date","Year","Month","Day","Hour","NordPool_DA_H_Vol(MWh)","NordPool_DA_H_Price(GBP/MWh)"])
for Year in Years:
    YearDFPrice = DA_Prices.loc[(DA_Prices["Date"].dt.year == Year)]
    YearDFVolume = DA_TurnOver.loc[(DA_TurnOver["Date"].dt.year == Year)]
    for Month in range(1,12+1):
        MonthDFPrice = YearDFPrice.loc[(YearDFPrice["Date"].dt.month == Month)]
        MonthDFVolume = YearDFVolume.loc[(YearDFVolume["Date"].dt.month == Month)]
        Days = monthrange(Year,Month)[1]
        for Day in range(1,Days+1):
            Years.set_description(f'{Year}-{Month}-{Day} ##>>')
            Date = datetime.date(Year,Month,Day)
            DayDFPrice = MonthDFPrice.loc[(MonthDFPrice["Date"].dt.day == Day)]
            DayDFVolume = MonthDFVolume.loc[(MonthDFVolume["Date"].dt.day == Day)]
            HH_IterCols = SearchColumns(Year,Month,Day)
            if (DayDFPrice.shape[0] == 1) and (DayDFVolume.shape[0] == 1):
                for SP, HHLabel in zip(range(1,len(HH_IterCols)+1),HH_IterCols):
                    DataList = []
                    Price = DayDFPrice[HHLabel].values[0]
                    Vol = DayDFVolume[HHLabel].values[0]
                    DataList.extend([Date,Year,Month,Day,SP,Vol,Price])
                    DataFrame.loc[len(DataFrame), DataFrame.columns] = (DataList)
DataFrame.to_csv("0.DataSource/NordPool/DA_Hourly/Hourly_PricesAndVolumes.csv",index=False)
DataFrame
'''

2024-4-30 ##>>: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


,Date,Year,Month,Day,Hour,NordPool_DA_H_Vol(MWh),NordPool_DA_H_Price(£/MWh)
0,2024-01-01,2024,1,1,1,11403.0,34.93
1,2024-01-01,2024,1,1,2,11716.0,46.6
2,2024-01-01,2024,1,1,3,11905.2,43.42
3,2024-01-01,2024,1,1,4,12569.5,18.03
4,2024-01-01,2024,1,1,5,13326.5,0.15
...,...,...,...,...,...,...,...
2898,2024-04-30,2024,4,30,20,13151.0,85.0
2899,2024-04-30,2024,4,30,21,13315.3,86.92
2900,2024-04-30,2024,4,30,22,12592.5,83.68
2901,2024-04-30,2024,4,30,23,12159.1,75.53


In [51]:
# genDemand = pd.read_csv("0.DataSource/National Demand/df_fuel_ckan_WithDemand.csv")
# genDemand['DATETIME'] = pd.to_datetime(genDemand['DATETIME'])
# genDemand = genDemand.resample('60min', on='DATETIME').mean()
# genDemand.to_csv("0.DataSource/National Demand/df_fuel_ckan_WithDemand_Hourly_SSP.csv")
# genDemand

C:\Users\swikr\AppData\Local\Temp\ipykernel_31736\1522817908.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  genDemand = genDemand.resample('60min', on='DATETIME').mean()


,SETTLEMENT_PERIOD,SBP(GBP/MWh),SSP(GBP/MWh),MID_APX_VOLUME(MWh),MID_APX_PRICE(GBP/MWh),MID_N2EX_VOLUME(MWh),MID_N2EX_PRICE(GBP/MWh),SSP(GBP/MWh).1,NIV(MWh),NordPool_HH_Vol(MWh),...,NET_INTCONNECTOR_FLOW,IFA_FLOW,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW
DATETIME,,,,,,,,,,,,,,,,,,,,,
2009-01-01 00:00:00+00:00,1.5,74.818820,57.190000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1841.5,2002.0,0.0,0.0,-160.5,0.0,0.0,NaN,NaN,NaN
2009-01-01 01:00:00+00:00,3.5,63.399510,45.354450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1727.0,1887.0,0.0,0.0,-160.0,0.0,0.0,NaN,NaN,NaN
2009-01-01 02:00:00+00:00,5.5,61.104645,48.255000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1593.5,1753.5,0.0,0.0,-160.0,0.0,0.0,NaN,NaN,NaN
2009-01-01 03:00:00+00:00,7.5,63.804375,37.535000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1594.5,1754.5,0.0,0.0,-160.0,0.0,0.0,NaN,NaN,NaN
2009-01-01 04:00:00+00:00,9.5,60.898855,37.820000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1594.5,1754.5,0.0,0.0,-160.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-29 19:00:00+00:00,39.5,87.000000,87.000000,NaN,NaN,NaN,NaN,NaN,NaN,162.00,...,7834.5,1999.5,993.0,1003.0,-450.0,-531.0,1019.0,1397.0,996.0,1408.0
2024-02-29 20:00:00+00:00,41.5,84.500000,84.500000,NaN,NaN,NaN,NaN,NaN,NaN,227.50,...,7951.0,1999.5,992.5,1003.0,-335.5,-529.0,1019.0,1397.0,996.5,1408.0
2024-02-29 21:00:00+00:00,43.5,78.959365,78.959365,NaN,NaN,NaN,NaN,NaN,NaN,157.50,...,7827.0,1999.0,992.0,1003.0,-438.0,-531.0,1000.0,1397.0,997.5,1407.5


In [46]:
# Hourly = pd.read_csv("0.DataSource/NordPool/DA_Hourly/Hourly_PricesAndVolumes.csv")
# Hourly['Date(UTC)'] = pd.to_datetime(Hourly['Date(UTC)'])
# Hourly.set_index("Date(UTC)",inplace=True)
# Hourly

,NordPool_DA_H_Vol(MWh),NordPool_DA_H_Price(£/MWh)
Date(UTC),,
2015-01-01 00:00:00+00:00,14917.0,40.79
2015-01-01 01:00:00+00:00,14582.1,39.61
2015-01-01 02:00:00+00:00,14127.2,37.18
2015-01-01 03:00:00+00:00,13317.8,32.40
2015-01-01 04:00:00+00:00,13916.3,29.29
...,...,...
2024-03-13 19:00:00+00:00,15311.7,81.39
2024-03-13 20:00:00+00:00,15668.9,74.13
2024-03-13 21:00:00+00:00,13920.7,64.50
